In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
#@title Import required modules

import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import SGDRegressor, LinearRegression

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae, convert_to_spherical
from networks import *
from posthoc import fitrlinear

In [ ]:
ids =                    np.load('../mpii_dataset_shared/loocv_ids.npy')
images = torch.as_tensor(np.load('../mpii_dataset_shared/loocv_images.npy'), dtype=torch.float)
hps =    torch.as_tensor(np.load('../mpii_dataset_shared/loocv_2d_hps.npy'), dtype=torch.float)
gazes =  torch.as_tensor(np.load('../mpii_dataset_shared/loocv_2d_gazes.npy'), dtype=torch.float)

In [ ]:
def mpii_chm_adj(ids, images, hps, gazes, looid, seed, path = '.', experiment_name = 'mpii_loocv', patience=10, device=torch.device('cuda:0')) :
    make_reproducibility(seed + looid)

    train_ids = np.concatenate([ids[:looid], ids[(looid+1):]]).reshape(-1)
    train_images = torch.cat([images[:looid], images[(looid+1):]]).reshape(-1,36,60)
    train_hps = torch.cat([hps[:looid], hps[(looid+1):]]).reshape(-1,2)
    train_gazes = torch.cat([gazes[:looid], gazes[(looid+1):]]).reshape(-1,2)
    train_y = train_gazes * 180 / np.pi

    test_ids = ids[looid]
    test_images = images[looid]
    test_hps = hps[looid]
    test_gazes = gazes[looid]
    test_y = test_gazes * 180 / np.pi

    # Load model
    # mean_model = ResNet_batchnorm.ResNet_batchnorm(out_features=2).to(device)
    # mean_model.load_state_dict(torch.load(f'{path}/results/corr_prec_{experiment_name}_{looid}_nhll_selected_mean_model.pt'))

    # Load trained parameters
    v_list = np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_v_list.npy')
    ind = np.max(np.where(v_list[:,0,0,0] !=0)[0]) - patience
    best_v_list = torch.as_tensor(v_list[ind], dtype=torch.float)

    test_Gamma = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_test_Gamma.npy')[2], dtype=torch.float)
    train_Gamma = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_train_Gamma.npy')[2], dtype=torch.float)

    w_list = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_w.npy'), dtype=torch.float)
    beta_list = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_beta.npy'), dtype=torch.float)

    # Load prediction
    prediction = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_pred.npy'), dtype=torch.float)
    prediction_adjusted = torch.as_tensor(np.load(f'{path}/prediction/corr_prec_{experiment_name}_{looid}_pred_adjusted.npy'), dtype=torch.float)


    # Recover fixed and random parts  (train dataset)
    train_fixed = train_Gamma @ beta_list[ind]
    train_random = torch.zeros_like(train_y)
    m = len(best_v_list)
    train_cluster= [np.where(train_ids == ids)[0] for ids in np.unique(train_ids)]

    for i in range(m) :
        train_random[train_cluster[i]] += train_Gamma[train_cluster[i]] @ best_v_list[i]

    test_fixed = convert_to_spherical(prediction[ind], deg=True)
    test_old_adj = convert_to_spherical(prediction_adjusted[ind], deg=True)

    # Check whether the index is found well
    # print(test_fixed - test_Gamma @ beta_list[ind])
    # print(test_fixed + test_Gamma @ w_list[ind] - test_old_adj)

    K = 2
    train_z = train_Gamma[:,1:]
    test_z = test_Gamma[:,1:]

    train_v_list = [torch.zeros_like(train_Gamma) for _ in range(K)]
    for i in range(m) :
        for k in range(K) :
            train_v_list[k][train_cluster[i]] = best_v_list[i][:,k].repeat(len(train_cluster[i]), 1)

    w_list = []
    for k in range(K) :
        w_list.append(fitrlinear(X = train_z.detach().cpu(), y = train_v_list[k].cpu(), device=device).to(device))
        w_list[k].fit(max_epoch=50)

    test_nu_list = [w_list[k].predict(test_z).detach().cpu() for k in range(K)]

    test_new_adj_1 = torch.zeros_like(test_gazes)
    test_new_adj_1[:,0] = torch.sum(test_Gamma * test_nu_list[0], dim = 1)
    test_new_adj_1[:,1] = torch.sum(test_Gamma * test_nu_list[1], dim = 1)

    test_mean_nu_list = [torch.mean(test_nu, dim=0) for test_nu in test_nu_list]
    test_new_adj_2 = torch.zeros_like(test_gazes)
    test_new_adj_2[:,0] = test_Gamma @ test_mean_nu_list[0]
    test_new_adj_2[:,1] = test_Gamma @ test_mean_nu_list[1]

    print(f'Test MAE (fixed) : {mae(test_fixed, test_y, is_3d=False, deg=True)}')
    print(f'Test MAE (old adjustment) : {mae(test_old_adj, test_y, is_3d=False, deg=True)}')
    print(f'Test MAE (new adjustment) : {mae(test_fixed + test_new_adj_1, test_y, is_3d=False, deg=True)}')
    print(f'Test MAE (new adjustment, averaged) : {mae(test_fixed + test_new_adj_2, test_y, is_3d=False, deg=True)}')

    return test_Gamma, test_nu_list, test_mean_nu_list, w_list, test_new_adj_1, test_new_adj_2

In [ ]:
# looid = 0
seed = 100
path = '.'
experiment_name = 'mpii_loocv'
patience=10
device=torch.device('cuda:0')

posthoc_list = []
for looid in range(15) :
    posthoc_list.append(mpii_chm_adj(ids, images, hps, gazes, looid, seed, path = path, experiment_name = experiment_name, patience=patience, device=device))

In [ ]:
nu_list = torch.stack([torch.stack(posthoc_list[looid][1]) for looid in range(15)])
mean_nu_list = torch.stack([torch.stack(posthoc_list[looid][2]) for looid in range(15)])

os.makedirs('./posthoc_results', exist_ok=True)
torch.save(nu_list, f'./posthoc_results/chm_mpii_loocv_nu.pt')
torch.save(mean_nu_list, f'./posthoc_results/chm_mpii_loocv_mean_nu.pt')